In [1]:
import DualwormFunctions as dw
import Observables as obs
%load_ext line_profiler
%load_ext memory_profiler
import RunBasis
import argparse
import StartStates as strst
import numpy as np

In [2]:
nst = 30;
nsm = 30;
nips = 20;
nb = 2;
measperiod = 1;

In [3]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = 28, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = 1000,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = 0,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = 0,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling')
    parser.add_argument('--h', type = float, default = 1.0,
                        help = 'Magnetic field')    
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--measperiod', type = int, default = measperiod,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = 8,
                        help = 'number of threads to use')

    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = 10,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--randominit', default = True, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = False, action = 'store_true',
                        help = '''initialise all the temperature with the maximally magnetised GS''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = True, action = 'store_true',
                        help = 'activate for single spin flip update')
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.05,60],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [4],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = True, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    #MAGNETIC FIELD PARAMETERS
    parser.add_argument('--h_list', nargs = '+', default = [0.0, 1.0],type = float,
                        help = 'list of limiting magnetic field values')
    parser.add_argument('--nh_list', nargs = '+', default = [5], type = int,
                        help = 'list of number of magnetic fields in between the given limiting temperatures')
    parser.add_argument('--stat_hfields_lims', nargs = '+', type = float,
                help = '''limiting magnetic fields for the various ranges of
                measurements''') 
    #CORRELATIONS PARAMETER
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default =True, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--charges', default = False, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = False, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = False, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = "testJ1hssf", help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [4]:
def callRunBasis():
    args = genArgs()
    return RunBasis.main(args)

In [5]:
meanstat, failedupdatesth, failedupdates = callRunBasis() 

Lattice side size:  28
J1  1000
J2  0
J3  0
J3st  0
Couplings extracted
Hamiltonian expression (without field) computed
Number of temperatures:  4
Number of magnetic fields:  5
Magnetic fields:  [0.   0.25 0.5  0.75 1.  ]
walker2params shape:  (20, 2)
walker2ids shape:  (20, 2)
ids2walker shape:  (4, 5)
walker2params =  [[2.00000000e+01 0.00000000e+00]
 [2.00000000e+01 2.50000000e-01]
 [2.00000000e+01 5.00000000e-01]
 [2.00000000e+01 7.50000000e-01]
 [2.00000000e+01 1.00000000e+00]
 [1.88207206e+00 0.00000000e+00]
 [1.88207206e+00 2.50000000e-01]
 [1.88207206e+00 5.00000000e-01]
 [1.88207206e+00 7.50000000e-01]
 [1.88207206e+00 1.00000000e+00]
 [1.77109762e-01 0.00000000e+00]
 [1.77109762e-01 2.50000000e-01]
 [1.77109762e-01 5.00000000e-01]
 [1.77109762e-01 7.50000000e-01]
 [1.77109762e-01 1.00000000e+00]
 [1.66666667e-02 0.00000000e+00]
 [1.66666667e-02 2.50000000e-01]
 [1.66666667e-02 5.00000000e-01]
 [1.66666667e-02 7.50000000e-01]
 [1.66666667e-02 1.00000000e+00]]
walker2ids =  [[0

ValueError: operands could not be broadcast together with shapes (2,) (5,) 

In [ ]:
print(failedupdatesth/(nst*nips))

In [ ]:
print(failedupdates/(nsm*nips*measperiod))